# Master Stock Analysis Summary

This notebook provides a summary of volume analysis and signals for all tracked stocks.

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn google-cloud-bigquery pandas-gbq plotly

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from google.cloud import bigquery
from datetime import datetime, timedelta
from IPython.display import HTML, display

# Set plot style
sns.set(style="darkgrid")
plt.rcParams['figure.figsize'] = (14, 8)

In [ ]:
# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Configuration
PROJECT_ID = "{PROJECT_ID}"  # Will be replaced with actual project ID
DATASET = "trading_insights"
TRACKED_STOCKS = {TRACKED_STOCKS_JSON}  # Will be replaced with actual tracked stocks

# Configuration file path (for updating tracked stocks)
CONFIG_PATH = "config/tracked_stocks.json"

## Data Import and Watchlist Management

This section allows you to import data for all tracked stocks and manage your watchlist.
You can update the tracked stocks and generate new notebooks for analysis.

In [ ]:
# Function to import data for all tracked stocks
def import_stock_data(stocks=None, days=365):
    """Import stock data from Alpha Vantage to BigQuery."""
    from google.cloud import bigquery
    import pandas as pd
    import requests
    import time
    from datetime import datetime, timedelta
    
    # Use all tracked stocks if none specified
    if stocks is None:
        stocks = []
        for direction in TRACKED_STOCKS:
            stocks.extend(TRACKED_STOCKS[direction])
    
    # Get Alpha Vantage API key
    api_key = input("Enter your Alpha Vantage API key: ")
    if not api_key:
        print("API key is required")
        return False
    
    # Initialize BigQuery client
    client = bigquery.Client(project=PROJECT_ID)
    
    # Import data for each stock
    for i, ticker in enumerate(stocks):
        print(f"Importing data for {ticker} ({i+1}/{len(stocks)})...")
        
        try:
            # Fetch daily data from Alpha Vantage
            url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&outputsize=full&apikey={api_key}"
            response = requests.get(url)
            data = response.json()
            
            if 'Time Series (Daily)' not in data:
                print(f"Error fetching data for {ticker}: {data.get('Note', 'Unknown error')}")
                # Wait for API rate limit
                time.sleep(12)  # 5 requests per minute = 12 seconds between requests
                continue
            
            # Convert to DataFrame
            df = pd.DataFrame.from_dict(data['Time Series (Daily)'], orient='index')
            
            # Map column names
            column_mapping = {
                '1. open': 'open',
                '2. high': 'high',
                '3. low': 'low',
                '4. close': 'close',
                '5. adjusted close': 'adjusted_close',
                '6. volume': 'volume',
                '7. dividend amount': 'dividend',
                '8. split coefficient': 'split_coefficient'
            }
            
            # Rename columns
            df = df.rename(columns=column_mapping)
            
            # Convert index to datetime
            df.index = pd.to_datetime(df.index)
            
            # Convert all columns to numeric
            for col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Add symbol column
            df['symbol'] = ticker
            
            # Reset index to make date a column
            df = df.reset_index()
            df = df.rename(columns={'index': 'date'})
            
            # Select only the columns we need
            columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'adjusted_close', 'symbol']
            df = df[columns]
            
            # Convert date to string (YYYY-MM-DD format)
            df['date'] = df['date'].dt.strftime('%Y-%m-%d')
            
            # Upload to BigQuery
            table_id = f"{DATASET}.stock_{ticker}_prices"
            
            # Use pandas_gbq to upload data
            df.to_gbq(
                destination_table=table_id,
                project_id=PROJECT_ID,
                if_exists='replace'
            )
            
            print(f"Successfully imported {len(df)} rows for {ticker}")
            
            # Wait for API rate limit
            time.sleep(12)  # 5 requests per minute = 12 seconds between requests
            
        except Exception as e:
            print(f"Error importing data for {ticker}: {str(e)}")
    
    print("Stock data import completed")
    return True

# Function to update tracked stocks
def update_tracked_stocks(buy_stocks=None, short_stocks=None):
    """Update the tracked stocks configuration and generate new notebooks."""
    import json
    import os
    import shutil
    from datetime import datetime
    from google.colab import files
    
    # Get current tracked stocks
    current_stocks = TRACKED_STOCKS.copy()
    
    # Update with new stocks if provided
    if buy_stocks is not None:
        current_stocks['buy'] = buy_stocks
    
    if short_stocks is not None:
        current_stocks['short'] = short_stocks
    
    # Create a backup of the current configuration
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_file = f"tracked_stocks_backup_{timestamp}.json"
    
    with open(backup_file, 'w') as f:
        json.dump(TRACKED_STOCKS, f, indent=2)
    
    # Download the backup file
    files.download(backup_file)
    
    # Update the configuration file
    with open(CONFIG_PATH, 'w') as f:
        json.dump(current_stocks, f, indent=2)
    
    print(f"Updated tracked stocks configuration. Backup saved to {backup_file}")
    
    # Generate new notebooks
    generate_notebooks(current_stocks)
    
    return current_stocks

# Function to generate notebooks
def generate_notebooks(tracked_stocks):
    """Generate individual stock analysis notebooks from template."""
    import os
    import json
    import re
    from pathlib import Path
    from google.colab import files
    
    # Create output directory if it doesn't exist
    output_dir = Path('notebooks')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Download template from GitHub if it doesn't exist locally
    template_path = Path('stock_analysis_template.ipynb')
    if not template_path.exists():
        !wget -q https://raw.githubusercontent.com/DanielDeenik/Delphi/feature/volume-analysis/notebooks/stock_analysis_template.ipynb -O stock_analysis_template.ipynb
    
    # Load template
    try:
        with open(template_path, 'r', encoding='utf-8') as f:
            template_content = f.read()
    except Exception as e:
        print(f"Error reading template file: {str(e)}")
        return False
    
    # Generate notebooks for each stock
    generated_notebooks = []
    
    for direction, tickers in tracked_stocks.items():
        for ticker in tickers:
            try:
                # Replace placeholders in template
                notebook_content = template_content
                notebook_content = notebook_content.replace('{TICKER}', ticker)
                notebook_content = notebook_content.replace('{PROJECT_ID}', PROJECT_ID)
                notebook_content = notebook_content.replace('{DIRECTION}', direction)
                
                # Replace other placeholders with dummy values (will be updated when notebook runs)
                notebook_content = notebook_content.replace('{SPIKE_COUNT}', '0')
                notebook_content = notebook_content.replace('{LATEST_Z_SCORE}', '0.0')
                notebook_content = notebook_content.replace('{CURRENT_SIGNAL}', 'NEUTRAL')
                
                # Save notebook
                output_path = output_dir / f"{ticker}_analysis.ipynb"
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(notebook_content)
                
                generated_notebooks.append(str(output_path))
                print(f"Generated notebook for {ticker}: {output_path}")
            
            except Exception as e:
                print(f"Error generating notebook for {ticker}: {str(e)}")
    
    # Create a zip file with all notebooks
    if generated_notebooks:
        !zip -j stock_notebooks.zip {' '.join(generated_notebooks)}
        files.download('stock_notebooks.zip')
    
    print(f"Generated notebooks for {len(tracked_stocks['buy']) + len(tracked_stocks['short'])} stocks")
    return True

In [ ]:
# Trigger data import for all tracked stocks
# Uncomment to run
# import_stock_data()

In [ ]:
# Update tracked stocks
# Example: update_tracked_stocks(buy_stocks=['AAPL', 'MSFT', 'GOOGL'], short_stocks=['BIDU', 'NIO', 'SNAP'])
# Uncomment to run
# update_tracked_stocks()

In [ ]:
# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

## Fetch Latest Analysis Results

Let's fetch the latest analysis results for all tracked stocks from BigQuery.

In [ ]:
# Fetch latest analysis results from master summary table
query = f"""
WITH LatestAnalysis AS (
  SELECT
    ticker,
    MAX(timestamp) AS latest_timestamp
  FROM
    `{PROJECT_ID}.{DATASET}.master_summary`
  GROUP BY
    ticker
)
SELECT
  s.*
FROM
  `{PROJECT_ID}.{DATASET}.master_summary` s
JOIN
  LatestAnalysis l
ON
  s.ticker = l.ticker
  AND s.timestamp = l.latest_timestamp
ORDER BY
  s.confidence DESC
"""

try:
    summary_df = client.query(query).to_dataframe()
    print(f"Fetched analysis results for {len(summary_df)} stocks")
except Exception as e:
    print(f"Error fetching analysis results: {str(e)}")
    # Create empty dataframe with expected columns
    summary_df = pd.DataFrame(columns=[
        'date', 'ticker', 'direction', 'signal', 'confidence',
        'volume_score', 'is_spike', 'spike_strength', 'notes', 'notebook_url', 'timestamp'
    ])

# If no data is available, create sample data for demonstration
if summary_df.empty:
    print("No data available in BigQuery. Creating sample data for demonstration.")
    
    # Create sample data
    sample_data = []
    
    # Add buy stocks
    for ticker in TRACKED_STOCKS['buy']:
        sample_data.append({
            'date': datetime.now().date(),
            'ticker': ticker,
            'direction': 'buy',
            'signal': np.random.choice(['STRONG_BUY', 'POTENTIAL_REVERSAL', 'NEUTRAL'], p=[0.3, 0.3, 0.4]),
            'confidence': np.random.uniform(0.5, 1.0),
            'volume_score': np.random.uniform(1.0, 3.0),
            'is_spike': np.random.choice([True, False], p=[0.3, 0.7]),
            'spike_strength': np.random.uniform(1.0, 2.0),
            'notes': f"Sample data for {ticker}",
            'notebook_url': f"https://colab.research.google.com/drive/your-notebook-id-for-{ticker}",
            'timestamp': datetime.now()
        })
    
    # Add short stocks
    for ticker in TRACKED_STOCKS['short']:
        sample_data.append({
            'date': datetime.now().date(),
            'ticker': ticker,
            'direction': 'short',
            'signal': np.random.choice(['STRONG_SHORT', 'POTENTIAL_REVERSAL', 'NEUTRAL'], p=[0.3, 0.3, 0.4]),
            'confidence': np.random.uniform(0.5, 1.0),
            'volume_score': np.random.uniform(1.0, 3.0),
            'is_spike': np.random.choice([True, False], p=[0.3, 0.7]),
            'spike_strength': np.random.uniform(1.0, 2.0),
            'notes': f"Sample data for {ticker}",
            'notebook_url': f"https://colab.research.google.com/drive/your-notebook-id-for-{ticker}",
            'timestamp': datetime.now()
        })
    
    summary_df = pd.DataFrame(sample_data)

# Display summary dataframe
summary_df.head()

## Dashboard: Signal Summary

Let's create a dashboard to visualize the signals for all tracked stocks.

In [ ]:
# Create a function to generate a styled HTML table
def generate_styled_table(df, title):
    # Create a copy of the dataframe with selected columns
    table_df = df[['ticker', 'direction', 'signal', 'confidence', 'volume_score', 'is_spike']].copy()
    
    # Add hyperlinks to ticker symbols
    table_df['ticker'] = table_df.apply(
        lambda row: f"<a href='{df.loc[row.name, 'notebook_url']}' target='_blank'>{row['ticker']}</a>",
        axis=1
    )
    
    # Format confidence as percentage
    table_df['confidence'] = table_df['confidence'].apply(lambda x: f"{x:.0%}")
    
    # Format volume score
    table_df['volume_score'] = table_df['volume_score'].apply(lambda x: f"{x:.2f}")
    
    # Format is_spike as Yes/No
    table_df['is_spike'] = table_df['is_spike'].apply(lambda x: "Yes" if x else "No")
    
    # Rename columns
    table_df.columns = ['Ticker', 'Direction', 'Signal', 'Confidence', 'Volume Z-Score', 'Volume Spike']
    
    # Convert to HTML
    html = f"<h2>{title}</h2>"
    html += "<style>"
    html += "table { border-collapse: collapse; width: 100%; }"
    html += "th, td { padding: 8px; text-align: left; border-bottom: 1px solid #ddd; }"
    html += "th { background-color: #f2f2f2; }"
    html += "tr:hover { background-color: #f5f5f5; }"
    html += "a { text-decoration: none; color: #0366d6; }"
    html += "a:hover { text-decoration: underline; }"
    html += "</style>"
    html += table_df.to_html(escape=False, index=False)
    
    return html

# Split dataframe by direction
buy_df = summary_df[summary_df['direction'] == 'buy'].sort_values('confidence', ascending=False)
short_df = summary_df[summary_df['direction'] == 'short'].sort_values('confidence', ascending=False)

# Generate styled tables
buy_table = generate_styled_table(buy_df, "Buy Candidates")
short_table = generate_styled_table(short_df, "Short Candidates")

# Display tables
display(HTML(buy_table))
display(HTML(short_table))

## Volume Analysis Visualization

Let's visualize the volume analysis results for all tracked stocks.

In [ ]:
# Create a scatter plot of volume score vs. confidence
fig = px.scatter(
    summary_df,
    x="volume_score",
    y="confidence",
    color="direction",
    size="spike_strength",
    hover_name="ticker",
    hover_data=["signal", "notes"],
    title="Volume Score vs. Confidence",
    labels={
        "volume_score": "Volume Z-Score",
        "confidence": "Signal Confidence",
        "direction": "Direction",
        "spike_strength": "Spike Strength"
    },
    color_discrete_map={"buy": "green", "short": "red"}
)

# Add ticker labels
fig.update_traces(textposition='top center')
fig.update_layout(
    height=600,
    width=900,
    xaxis=dict(title="Volume Z-Score", range=[0, max(summary_df['volume_score']) * 1.1]),
    yaxis=dict(title="Signal Confidence", range=[0, 1.1])
)

fig.show()

## Signal Distribution

Let's visualize the distribution of signals across all tracked stocks.

In [ ]:
# Count signals by type and direction
signal_counts = summary_df.groupby(['direction', 'signal']).size().reset_index(name='count')

# Create a grouped bar chart
fig = px.bar(
    signal_counts,
    x="signal",
    y="count",
    color="direction",
    title="Signal Distribution",
    labels={
        "signal": "Signal Type",
        "count": "Count",
        "direction": "Direction"
    },
    color_discrete_map={"buy": "green", "short": "red"}
)

fig.update_layout(
    height=500,
    width=800,
    xaxis=dict(title="Signal Type"),
    yaxis=dict(title="Count")
)

fig.show()

## Top Signals

Let's identify the top signals for both buy and short directions.

In [ ]:
# Function to display top signals
def display_top_signals(df, direction, n=3):
    # Filter by direction and non-neutral signals
    filtered_df = df[(df['direction'] == direction) & (df['signal'] != 'NEUTRAL')]
    
    # Sort by confidence
    top_df = filtered_df.sort_values('confidence', ascending=False).head(n)
    
    if len(top_df) == 0:
        print(f"No {direction} signals found")
        return
    
    print(f"Top {n} {direction.capitalize()} Signals:")
    for i, (_, row) in enumerate(top_df.iterrows(), 1):
        print(f"{i}. {row['ticker']} - {row['signal']} (Confidence: {row['confidence']:.0%})")
        print(f"   Volume Z-Score: {row['volume_score']:.2f}")
        print(f"   Notes: {row['notes']}")
        print(f"   Notebook: {row['notebook_url']}")
        print()

# Display top buy signals
display_top_signals(summary_df, 'buy')

# Display top short signals
display_top_signals(summary_df, 'short')

## Summary and Next Steps

This notebook provides a summary of volume analysis and signals for all tracked stocks.

### Key Findings:
- Number of buy signals: {BUY_SIGNAL_COUNT}
- Number of short signals: {SHORT_SIGNAL_COUNT}
- Top buy candidate: {TOP_BUY_TICKER}
- Top short candidate: {TOP_SHORT_TICKER}

### Next Steps:
1. Review individual notebooks for stocks with strong signals
2. Monitor volume patterns for potential trade opportunities
3. Update analysis daily to track changes in volume patterns